# Simulación 2 - Call Center Metrogas

## Contexto
Se dispone de 3 tipos de puestos, y diferentes cantidades de cada uno de ellos:

- Emergencia (E)
- Comercial (C)
- Polifuncional (P)

Y dos colas:

- Cola para emergencia (NSE)
- Cola para comercial (NSC)

Los puestos comerciales y los puestos de emergencia solo atienden su propia cola. Los puestos polifuncionales atienden solo cuando todos los puestos comerciales y/o todos los puestos de emergencia estan ocupados y hay gente esperando en la cola para ser atendido,sera atendido por un polifuncional, dandole mas prioridad a los que estan en la cola de emergencia 

## Importando librerias de simpy

In [1]:
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections
from itertools import combinations_with_replacement
import xlwt 
from xlwt import Workbook 
import time

_____

## Simulacion de una llegada

In [2]:
def distribucion_llegada(env,puestos,cont):
    while True:
    #for i in range(n):
    
        cont["NT"] += 1
        
        # Genero un cliente random 
        r = random.random()
        #cont['file'].write("Distribucion de generacion de tipo de cliente: %5.2f \n"%r)
        
        
        # Si genere un cliente para emergencia
        if r <= cont["PE"]:
            
            ce = Customer(name = "clienteE%02d"%cont["NT"])
            
            # Genero IA para el siguiente cliente y genero el timeout para su llegada 
            ia = generar_IA(cont)
            yield env.timeout(ia)
            cont['file'].write("T=%5.2f %s: COLA EMERGENCIA - llegue con un IA: %5.2f \n"%(env.now,ce.name,ia))
            colas(puestos,cont) # Imprimo el estao de las colas
            
            # Genero la llamada
            env.process(ce.llamada_emergencia(env,puestos,cont))
            
        # Si genere un cliente para comercial    
        else: 
            
            cc = Customer(name = "clienteC%02d"%cont["NT"])
            # Genero IA para el siguiente cliente y genero el timeout para su llegada 
            ia = generar_IA(cont)
            yield env.timeout(ia)
            cont['file'].write("T=%5.2f %s: COLA COMERCIAL - llegue con un IA: %5.2f \n"%(env.now,cc.name,ia))
            colas(puestos,cont) # Imprimo el estao de las colas
            
            # Genero la llamada
            env.process(cc.llamada_comercial(env,puestos,cont))

In [3]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(self,env,puestos,cont):
        
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        # puestos = [comercial, emergencia, polifuncional]
        emergencia, polifuncional = puestos[1].request(priority=0), puestos[2].request(priority=0)
        atendido = yield emergencia | polifuncional 
        #assert atendido == {emergencia}
        
        # Me atendieron por puestos polifuncionales?
        if polifuncional in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            cont['file'].write("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto polifuncional despues de esperar %5.2f \n"%(env.now,self.name,wait))
            colas(puestos,cont)
            cont['file'].write("Tiempo de Atencion generado para %s: %5.2f \n"%(self.name,ta))
            yield env.timeout(ta)
            cont['file'].write("T=%5.2f, %s: Sali de un puesto de polifuncional \n"%(env.now,self.name))
        
        # Me atendieron por un puesto de emergencia
        else:
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                cont["NECE"] += 1
            
            ta = generar_TA(cont)
            cont['file'].write("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar %5.2f \n"%(env.now,self.name,wait))
            colas(puestos,cont)
            cont['file'].write("Tiempo de Atencion generado para %s: %5.2f \n"%(self.name,ta))
            yield env.timeout(ta)
            cont['file'].write("T=%5.2f, %s: Sali de un puesto de emergencia \n"%(env.now,self.name))
            
            
    def llamada_comercial(self,env,puestos,cont):
        
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        # puestos = [comercial, emergencia, polifuncional]
        comercial, polifuncional = puestos[0].request(priority=0), puestos[2].request(priority=1) # comercial tn menos prioridad q emergencia
        atendido = yield comercial | polifuncional 
        
        # Me atendieron por puestos polifuncionales?
        if polifuncional in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            cont['file'].write("T=%5.2f, %s: COLA COMERCIAL - Me atendieron en un puesto polifuncional despues de esperar %5.2f \n"%(env.now,self.name,wait))
            colas(puestos,cont)
            cont['file'].write("Tiempo de Atencion generado para %s: %5.2f \n"%(self.name,ta))
            yield env.timeout(ta)
            cont['file'].write("T=%5.2f, %s: Sali de un puesto de polifuncional \n"%(env.now,self.name))
        
        # Me atendieron por un puesto comercial
        else:
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NECC"] += 1
                cont["NEC"] += 1
            
            ta = generar_TA(cont)
            cont['file'].write("T=%5.2f, %s: COLA COMERCIAL - Me atendieron en un puesto comercial despues de esperar %5.2f \n"%(env.now,self.name,wait))
            colas(puestos,cont)
            cont['file'].write("Tiempo de Atencion generado para %s: %5.2f \n"%(self.name,ta))
            yield env.timeout(ta)
            cont['file'].write("T=%5.2f, %s: Sali de un puesto de comercial \n"%(env.now,self.name))

In [4]:
def colas(p,cont):
    cont['file'].write("Atendiendo:[c:%1d, e:%1d, p:%1d] \n"%(p[0].count,p[1].count,p[2].count))
    cont['file'].write("En cola:[c:%1d, e:%1d] \n"%(len(p[0].queue),len(p[1].queue)))


In [5]:
def generar_TA(cont):
    r = 0

    # Mientras caiga fuera de esta zona, generame otro
    while r < 52.2 or r > 439.2:

        try:
            r = int(random.lognormvariate(cont["muTA"],cont["sigmaTA"]))
        except OverflowError:
            r = float('inf')
    
    return r

In [6]:
def generar_IA(cont):
    r = 0

    while r <= 0 or r > 52.36:

        try:
            r = int(random.lognormvariate(cont["muIA"],cont["sigmaIA"]))
        except OverflowError:
            r = float('inf')
    
    return r

## Inicializacion de la simulacion

In [7]:
def simular(cantCom,cantEmer,cantPolif,ci,maxTime):
    env = simpy.Environment()
    puestos = [simpy.PriorityResource(env,capacity=cantCom),
           simpy.PriorityResource(env,capacity=cantEmer),
           simpy.PriorityResource(env,capacity=cantPolif)] # menor el numero priority, mayor es la prioridad
           
    env.process(distribucion_llegada(env,puestos,ci))
    env.run(until=maxTime)

## Condiciones Iniciales

In [8]:
# Experimental data
NT = 0
maxTime = 500 #segundos
#maxClient = 10
#random.seed(RANDOM_SEED)
PE = 0.0619 # Porcentaje Emergencia
PC = 0.9381 # Porcentaje Comercial
NEC = 0 # Cantidad de gente que espero en cola menor a 40 segundos
NECC = 0
NECE = 0
PEC = 0.0 # Porcentaje de espera en cola menor a 40 segundos
file = None

# TA - distribucion normal
muTA = 245.712
sigmaTA = 193.499

# IA - distribucion normal
muIA = 9.585 
sigmaIA = 42.785

# Contadores
ci = {"multitasking":0,
     "muTA":muTA,
     "sigmaTA":sigmaTA,
     "muIA":muIA,
     "sigmaIA":sigmaIA,
     "PE":PE,
     "PC":PC,
     "emer":0,
     "com":0,
     "pol":0,
     "NEC":NEC,
     "NECC":NECC,
     "NECE":NECE,
     "NT":NT,
     "file":file}

## Ingreso manual de Puestos de atencion - variables de control

!! Generar una version para automatizar esto, que genere diferentes combinaciones dado un rango

Y tmbn dejar la opcion de generarlo manualmente


print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
ci["com"] = cantCom
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())
ci["emer"] = cantEmer
print('Ingrese cantidad de puestos polifuncionales')
cantPolif = int(input())
ci["pol"] = cantPolif

simular(cantCom,cantEmer,cantPolif,ci,maxTime)


## Ejecucion automatica con combinaciones de variables de control

In [9]:
# Create an Excel to record the results 
wb = Workbook() 
t = time.localtime()
current_time = time.strftime("%Hh%Mm", t)
sheet = wb.add_sheet(str(current_time)) 
sheet.write(0, 0, 'Emergencia')
sheet.write(0, 1, 'Comercial')
sheet.write(0, 2, 'Polifuncional')
sheet.write(0, 3, 'NT')
sheet.write(0, 4, 'NEC')
sheet.write(0, 5, 'PEC')
simulacion = 0

# Create a text file to record processing log
file = open("log_sim2.txt","w")
ci['file'] = file

n, k = [10,20,30,40,50,60,70,80,90,100], 3
cant_puestos = list(combinations_with_replacement(n,k))

for i in cant_puestos:
    
    cantEmer = i[0]
    cantCom = i[1]
    cantPolif = i[2]
    simular(cantCom,cantEmer,cantPolif,ci,maxTime)
    simulacion += 1
        
    sheet.write(simulacion, 0, cantEmer)
    sheet.write(simulacion, 1, cantCom)
    sheet.write(simulacion, 2, cantPolif)
    sheet.write(simulacion, 3, ci["NT"]) # Cantidad de gente total
    sheet.write(simulacion, 4, ci["NEC"]) # Cantidad de gente que haya esperado menor a 40 segundos
    PEC = (ci["NEC"] / ci["NT"]) * 100
    print('Puestos: E:%1d C:%1d P:%1d \n'%(cantEmer,cantCom,cantPolif))
    print('PEC: %1d' %PEC,'%')
    sheet.write(simulacion, 5, PEC)
    file.write("================================================================================ \n") 
    file.write("\n")
    
        
wb.save('resultados_Simulacion2.xls')
file.close()
print('OK')        

Puestos: E:10 C:10 P:10 

PEC: 26 %
Puestos: E:10 C:10 P:20 

PEC: 45 %
Puestos: E:10 C:10 P:30 

PEC: 61 %
Puestos: E:10 C:10 P:40 

PEC: 69 %
Puestos: E:10 C:10 P:50 

PEC: 76 %
Puestos: E:10 C:10 P:60 

PEC: 79 %
Puestos: E:10 C:10 P:70 

PEC: 82 %
Puestos: E:10 C:10 P:80 

PEC: 84 %
Puestos: E:10 C:10 P:90 

PEC: 85 %
Puestos: E:10 C:10 P:100 

PEC: 87 %
Puestos: E:10 C:20 P:20 

PEC: 84 %
Puestos: E:10 C:20 P:30 

PEC: 83 %
Puestos: E:10 C:20 P:40 

PEC: 84 %
Puestos: E:10 C:20 P:50 

PEC: 85 %
Puestos: E:10 C:20 P:60 

PEC: 86 %
Puestos: E:10 C:20 P:70 

PEC: 87 %
Puestos: E:10 C:20 P:80 

PEC: 88 %
Puestos: E:10 C:20 P:90 

PEC: 88 %
Puestos: E:10 C:20 P:100 

PEC: 89 %
Puestos: E:10 C:30 P:30 

PEC: 89 %
Puestos: E:10 C:30 P:40 

PEC: 89 %
Puestos: E:10 C:30 P:50 

PEC: 89 %
Puestos: E:10 C:30 P:60 

PEC: 90 %
Puestos: E:10 C:30 P:70 

PEC: 90 %
Puestos: E:10 C:30 P:80 

PEC: 90 %
Puestos: E:10 C:30 P:90 

PEC: 91 %
Puestos: E:10 C:30 P:100 

PEC: 91 %
Puestos: E:10 C:40 P:40 
